# 식신 - 양재역으로 검색
- 명칭, 주요메뉴, 평점, 전화번호, 주소

In [1]:
 import requests
 import pandas as pd
 from urllib.parse import quote

- 데이터 가져오기

In [2]:
base_url = 'https://www.siksinhot.com'
url = f'{base_url}/search?keywords={quote("양재역")}'
req = requests.get(url)
html = req.text
#html

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

- 찾으려고 하는 데이터의 태그 찾기

In [5]:
lis = soup.select('div.listTy1 > ul > li')
len(lis)

12

- 여러 개의 데이터 중 하나를 선택해서 원하는 정보 추출

In [6]:
li = lis[0]
href = li.select_one('a')['href']
href

'/P/358859'

In [7]:
url = base_url + href
req = requests.get(url)
sub_soup = BeautifulSoup(req.text, 'html.parser')

In [8]:
store = sub_soup.select_one('.title')
info = store.find('h3').get_text()
info

'그릭슈바인 양재역점3.6HOT'

In [9]:
score = store.select_one('h3 > strong').string
score

'3.6'

In [10]:
info.find(score)        # info 문자열에서 score위치(인덱스)를 알아내는 메서드

10

In [11]:
name = info[:info.find(score)]
name

'그릭슈바인 양재역점'

In [12]:
menu = sub_soup.select('.store_info p')[1].get_text()
menu

'나이트라이프 > 맥주/호프'

In [13]:
tel = sub_soup.select_one('.p_tel p').get_text()
tel

'02-572-6100'

In [14]:
addr = sub_soup.select_one('.txt_adr').string
addr

'서울특별시 서초구 강남대로 224'

- 한 페이지에 있는 모든 데이터를 반복문으로 가져오기

In [26]:
name_list, score_list, menu_list, tel_list, addr_list = [],[],[],[],[]
for i, li in enumerate(lis):
    href = li.select_one('a')['href']
    name = li.select_one('.store').string
    score = li.select_one('em').string
    if score == '평가중':
        score = '0'
    menu = li.select_one('.cnt p').get_text()

    url = base_url + href
    req = requests.get(url)
    sub_soup = BeautifulSoup(req.text, 'html.parser')

    try:
        tel = sub_soup.select_one('.p_tel p').get_text()
        addr = sub_soup.select_one('.txt_adr').string

        name_list.append(name)
        score_list.append(float(score))
        menu_list.append(menu)
        tel_list.append(tel)
        addr_list.append(addr)
    except:
        print(i)

In [27]:
df = pd.DataFrame({
    '식당명':name_list, '평점':score_list, '메뉴':menu_list,
    '전화번호':tel_list, '주소':addr_list
})
df

,식당명,평점,메뉴,전화번호,주소
0,그릭슈바인 양재역점,3.6,"그릭슈바인bbq, 슈바이네 학센, 모듬 소세지 bbq, 콥 샐러드",02-572-6100,서울특별시 서초구 강남대로 224
1,미스터브리즈 양재역점,3.4,"아메리카노, 허니버터브레드, 와플",02-576-4333,서울 서초구 강남대로34길 6
2,동신참치 양재역점,3.0,"참치회, 참치정식",02-529-7130,서울 서초구 강남대로 213
3,새우닭 양재역점,0.0,"딱새우회, 닭도리탕, 딱새우튀김, 술국닭계장, 마라해물새우",02-575-6088,서울특별시 서초구 남부순환로356길 52 1층
4,양재역 옛날짜장,0.0,"짜장면, 짬뽕, 볶음밥, 고추잡채, 쟁반짜장, 탕수육",031-503-2550,경기도 시흥시 공단1대로 13
5,영동족발 3호점,4.1,"족발大, 쟁반막국수大, 순두부찌개, 족발 중, 해물파전",02-575-0250,서울특별시 서초구 남부순환로 2628-18
6,황재벌,3.9,"꼼장어, 쭈꾸미, 왕 꼼장어, 닭발, 계란찜, 알 주먹밥",02-3471-5454,서울특별시 서초구 남부순환로347길 42-4 신화빌딩
7,김영모과자점 도곡타워점,4.1,"몽블랑, 바게트샌드위치, 레즌 천연발효빵, 생크림앙팡, 마늘바게트",02-3460-2005,서울특별시 강남구 언주로30길 10 현대비젼21
8,소호정 본점,4.0,"안동국시, 국밥, 수육(대), 전(대), 묵(대)",02-579-7282,서울특별시 서초구 논현로 27
9,산동칼국수,3.9,"손칼국수, 만두국, 왕만두",02-3473-7972,서울특별시 서초구 강남대로37길 63


- 문제 발생한 업소 살펴보기

In [28]:
li = lis[0]
li.select_one('.store').string

'그릭슈바인 양재역점'

In [29]:
li.select_one('.score').string

'3.6'

In [30]:
li.select_one('.cnt p').get_text()

'그릭슈바인bbq, 슈바이네 학센, 모듬 소세지 bbq, 콥 샐러드'